In [1]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

True

In [5]:
#Read OPENAI_API_KEY from environment variable
openai_api_key = os.environ['OPENAI_API_KEY']
embeddings = OpenAIEmbeddings(disallowed_special=())

In [6]:
#Login to ActiveLoop using API Key environment variable
active_loop_api_key = os.environ['ACTIVE_LOOP_API_KEY']
!activeloop login -t $active_loop_api_key

Successfully logged in to Activeloop.


In [7]:
import os
from langchain.document_loaders import TextLoader

root_dir = 'repo'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try:
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except:
            pass

In [8]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

Created a chunk of size 1535, which is longer than the specified 1000
Created a chunk of size 1673, which is longer than the specified 1000
Created a chunk of size 1390, which is longer than the specified 1000
Created a chunk of size 1115, which is longer than the specified 1000
Created a chunk of size 1565, which is longer than the specified 1000
Created a chunk of size 1502, which is longer than the specified 1000
Created a chunk of size 1432, which is longer than the specified 1000
Created a chunk of size 1205, which is longer than the specified 1000
Created a chunk of size 1332, which is longer than the specified 1000
Created a chunk of size 2442, which is longer than the specified 1000
Created a chunk of size 1111, which is longer than the specified 1000
Created a chunk of size 3002, which is longer than the specified 1000
Created a chunk of size 1797, which is longer than the specified 1000
Created a chunk of size 1168, which is longer than the specified 1000
Created a chunk of s

In [9]:
username = os.environ['username']
db = DeepLake(
    dataset_path=f"hub://{username}/jockmkt",
    embedding_function=embeddings,
)
db.add_documents(texts)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-7fbpd8wLY1Ik8V4ss5aIkC6x on tokens per min. Limit: 1000000 / min. Current: 760247 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-7fbpd8wLY1Ik8V4ss5aIkC6x on tokens per min. Limit: 1000000 / min. Current: 681002 / min. Contact us through our help center at help.openai.com if you continue to have issues..
|

Dataset(path='hub://bhagirathbhardwaj/jockmkt', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype       shape       dtype  compression
  -------    -------     -------     -------  ------- 
 embedding  embedding  (2219, 1536)  float32   None   
    id        text      (2219, 1)      str     None   
 metadata     json      (2219, 1)      str     None   
   text       text      (2219, 1)      str     None   


['9ca52c64-1e74-11ee-b183-eed7f53b3186',
 '9ca52d5e-1e74-11ee-b183-eed7f53b3186',
 '9ca52dae-1e74-11ee-b183-eed7f53b3186',
 '9ca52dd6-1e74-11ee-b183-eed7f53b3186',
 '9ca52e08-1e74-11ee-b183-eed7f53b3186',
 '9ca52e30-1e74-11ee-b183-eed7f53b3186',
 '9ca52e62-1e74-11ee-b183-eed7f53b3186',
 '9ca52eee-1e74-11ee-b183-eed7f53b3186',
 '9ca52f2a-1e74-11ee-b183-eed7f53b3186',
 '9ca52f5c-1e74-11ee-b183-eed7f53b3186',
 '9ca52f84-1e74-11ee-b183-eed7f53b3186',
 '9ca52fac-1e74-11ee-b183-eed7f53b3186',
 '9ca52fd4-1e74-11ee-b183-eed7f53b3186',
 '9ca52ffc-1e74-11ee-b183-eed7f53b3186',
 '9ca53024-1e74-11ee-b183-eed7f53b3186',
 '9ca53042-1e74-11ee-b183-eed7f53b3186',
 '9ca5306a-1e74-11ee-b183-eed7f53b3186',
 '9ca53092-1e74-11ee-b183-eed7f53b3186',
 '9ca530ba-1e74-11ee-b183-eed7f53b3186',
 '9ca530e2-1e74-11ee-b183-eed7f53b3186',
 '9ca5310a-1e74-11ee-b183-eed7f53b3186',
 '9ca53132-1e74-11ee-b183-eed7f53b3186',
 '9ca53164-1e74-11ee-b183-eed7f53b3186',
 '9ca5318c-1e74-11ee-b183-eed7f53b3186',
 '9ca531aa-1e74-

In [10]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 10

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name="gpt-3.5-turbo")
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [15]:
# Read questions from the text file
with open('questions.txt', 'r') as file:
    questions = file.read().splitlines()

chat_history = []

for index, question in enumerate(questions):
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    #print(f"**Answer**: {result['answer']} \n")
    # Write the answer to a separate file
    answer_filename = f'answers/answer{index+1}.md'
    with open(answer_filename, 'w') as answer_file:
        answer_file.write(f"# Answer to Question {index+1}\n\n")
        answer_file.write(result["answer"])

-> **Question**: There are two repos within this directory, one for SDK and the other with an implementation of a trading bot. Explain these to us, 

**Answer**: Sure! Within this directory, there are two repos:

1. **jockmkt_sdk**: This is the software development kit (SDK) for JockMKT. It provides the necessary tools and functions for interacting with the JockMKT API, including authentication, retrieving player information, placing orders, and accessing market data. The SDK simplifies the process of building trading strategies by handling the underlying API calls and data management.

2. **jockbot**: This is an implementation of a trading bot using the JockMKT SDK. The JockBot is designed to make it easier for users to implement and edit electronic trading strategies. It handles websocket communication, data storage, and organization structure, allowing users to focus on building their own strategy classes. The repo includes multiple prebuilt trading strategies for both IPO and live 